# 作成したモデルのdeployとserving

０１_image_classificationで作ったモデルを今度はAPIとして使用できるようにします。

google ai-platformでは作成したmodelからendpointの作成を行う機能があるため、それを使ってみます。  
endpointの作成は2つの処理にわかれていて、modelの作成とversionの作成をおこなう必要があります。

%%bash

USER=hdtkkj
BUCKET=mixi-newbie-2020
VERSION=001

gcloud ml-engine models create mfashion_${USER} --regions asia-northeast1
gcloud ml-engine versions create mfashion_${USER}_${VERSION} \
  --model mfashion_${USER} \
  --origin gs://${BUCKET}/${USER}/${VERSION} \
  --runtime-version 2.1 \
  --python-version 3.7

versionの作成が完了したら、このendpointにリクエストを投げてみましょう

リクエストは同一のproject内からを想定し、google-api-clientを使用します。

In [ ]:
!pip install google-api-python-client 

In [ ]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):

    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
(X_train_orig, y_train_orig), (X_test_orig, y_test_orig) = fashion_mnist.load_data()
requests=X_test_orig[0:8]
requests = np.expand_dims(requests, -1)
requests = requests / 255.0

predict_json("tanpopo-ml","mfashion_hdtkkj", requests.tolist())